# Stable Diffusion Image Generation using noobai-XL-1.1

Created by: Vladus-CPU
Last Updated: 2025-01-01

This notebook demonstrates how to generate images using noobai-XL-1.1 model with optimized parameters for high-quality image generation.

## Installing Required Libraries

In [ ]:
!pip install --upgrade diffusers transformers torch torchvision matplotlib safetensors accelerate

## Importing Required Modules

In [ ]:
from diffusers import StableDiffusionXLPipeline, EulerAncestralDiscreteScheduler, DDIMScheduler
from transformers import CLIPTextModel, CLIPTokenizer
import torch
from PIL import Image
import matplotlib.pyplot as plt
import random

## GPU Availability Check

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Pipeline Loading Function with Sampling Method Selection

In [ ]:
def load_pipeline(scheduler_type="Euler"):
    """Load the noobai-XL-1.1 pipeline with specified scheduler.
    
    Args:
        scheduler_type (str): Either 'Euler' or 'DDIM'
        
    Returns:
        StableDiffusionXLPipeline: The loaded pipeline
    """
    model_id = "Laxhar/noobai-XL-1.1"
    
    if scheduler_type == "Euler":
        scheduler = EulerAncestralDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
    elif scheduler_type == "DDIM":
        scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler")
    else:
        raise ValueError("Unknown scheduler type")
    
    pipe = StableDiffusionXLPipeline.from_pretrained(
        model_id,
        scheduler=scheduler,
        torch_dtype=torch.float16 if device.type == "cuda" else torch.float32,
        safety_checker=None,
        variant="fp16" if device.type == "cuda" else None
    )
    
    # Enable memory efficient attention if available
    if hasattr(pipe, "enable_xformers_memory_efficient_attention"):
        pipe.enable_xformers_memory_efficient_attention()
    
    pipe = pipe.to(device)
    return pipe

## Image Generation Function

In [ ]:
def generate_image(
    prompt,
    negative_prompt="",
    sampler="Euler",
    sampling_steps=30,
    seed=None,
    cfg_scale=7.0,
    image_size=(1024, 1024),
    hires_fix=False,
    hires_scale=1.5,
    hires_steps=20
):
    """Generate an image using noobai-XL-1.1 model.
    
    Args:
        prompt (str): Text description of the desired image
        negative_prompt (str): Things to avoid in the image
        sampler (str): Sampling method ('Euler' or 'DDIM')
        sampling_steps (int): Number of sampling steps
        seed (int, optional): Random seed for reproducibility
        cfg_scale (float): Text guidance scale
        image_size (tuple): Output image dimensions (width, height)
        hires_fix (bool): Whether to apply high-resolution fix
        hires_scale (float): Scale factor for high-resolution fix
        hires_steps (int): Number of steps for high-resolution fix
    
    Returns:
        PIL.Image.Image: Generated image
    """
    # Set seed for reproducibility
    if seed is not None:
        torch.manual_seed(seed)
        random.seed(seed)
        generator = torch.Generator(device=device).manual_seed(seed)
    else:
        generator = None
    
    # Load pipeline
    pipe = load_pipeline(sampler)
    
    # Generate image
    output = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=sampling_steps,
        guidance_scale=cfg_scale,
        height=image_size[1],
        width=image_size[0],
        generator=generator
    )
    
    image = output.images[0]
    
    if hires_fix:
        # Calculate new dimensions
        new_width = int(image_size[0] * hires_scale)
        new_height = int(image_size[1] * hires_scale)
        
        # Generate high-resolution image
        output = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=hires_steps,
            guidance_scale=cfg_scale,
            height=new_height,
            width=new_width,
            generator=generator
        )
        
        image = output.images[0]
    
    return image

## Example Usage

In [ ]:
# Example prompt
prompt = "masterpiece, best quality, ultra detailed, A majestic dragon soaring through storm clouds, lightning crackling around its wings, highly detailed scales, dynamic pose, epic lighting"
negative_prompt = "worst quality, low quality, normal quality, lowres, low details, oversaturated, undersaturated, overexposed, underexposed, grainy, blur, text, watermark, signature, error"

# Configure generation parameters
params = {
    'negative_prompt': negative_prompt,
    'sampler': "Euler",
    'sampling_steps': 30,
    'seed': 42,
    'cfg_scale': 7.0,
    'image_size': (1024, 1024),
    'hires_fix': True,
    'hires_scale': 1.5,
    'hires_steps': 20
}

# Generate image
generated_image = generate_image(prompt, **params)

# Display the generated image
plt.figure(figsize=(15, 15))
plt.imshow(generated_image)
plt.axis('off')
plt.show()

## Save Generated Image

In [ ]:
# Save the generated image with timestamp
from datetime import datetime
timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_path = f"generated_image_{timestamp}.png"
generated_image.save(output_path)
print(f"Image saved to {output_path}")